In [3]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [4]:
data_frame = pd.read_csv('data.csv', encoding='latin-1')
tweet_series = data_frame['Tweet']
print("Banyak Data Tweet = {}".format(len(tweet_series)))

label = []
for index in range(len(tweet_series)):
    temp_label = 0
    for column in data_frame:
        if 'Tweet' not in column:
            temp_label = temp_label or data_frame[column][index]
    label.append(temp_label)

print("Jumlah Label = {}".format(len(label)))

Banyak Data Tweet = 38922
Jumlah Label = 38922


In [5]:
tweets = tweet_series
labels = label

print(type(tweets))
print(type(labels))

print("Terdapat {} Data".format(len(tweets)))

<class 'pandas.core.series.Series'>
<class 'list'>
Terdapat 38922 Data


In [6]:
VOCAB_SIZE = 1000
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token = '<00V>')
tokenizer.fit_on_texts(tweets)
word_index = tokenizer.word_index
print(len(word_index))
# print(word_index)

39922


In [7]:
sequences = tokenizer.texts_to_sequences(tweets)
print(tweets[0])
print(sequences[0])

kebijakan kayak gini nyape nyapein doang, percuma banget
[922, 263, 398, 1, 1, 452, 1, 49]


In [8]:
list_length = [len(x) for x in sequences]
SEQUENCES_LENGTH = max(list_length)
print("Maksimal Panjang Sequences = {}".format(SEQUENCES_LENGTH))

Maksimal Panjang Sequences = 129


In [9]:
padded_sequences = pad_sequences(sequences, padding = 'post', maxlen = SEQUENCES_LENGTH)
print(padded_sequences[0])

[922 263 398   1   1 452   1  49   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0]


In [8]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
training_portion = 0.95 * len(label)
training_portion = int(training_portion)
labels = np.array(labels)
train_data = padded_sequences[:training_portion]
train_label = labels[:training_portion]
val_data = padded_sequences[training_portion:]
val_label = labels[training_portion:]

print(len(train_data))
print(len(train_label))
print(len(val_data))
print(len(val_label))

36975
36975
1947
1947


In [71]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 123),
    tf.keras.layers.Conv1D(64, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model1.summary()

model1.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history_adam = model1.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

KeyboardInterrupt: 

In [9]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 123),
    tf.keras.layers.Conv1D(64, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model2.summary()

model2.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history2 = model2.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 50,
    verbose = 1
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 123, 500)          500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 116, 64)           256064    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 58, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 58, 64)            0         
_________________________________________________________________
dense (Dense)                (None, 58, 500)           32500     
_________________________________________________________________
dropout_1 (Dropout)          (None, 58, 500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 58, 100)           5

KeyboardInterrupt: 

In [11]:
model22 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 123),
    tf.keras.layers.Conv1D(64, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation = 'relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model22.summary()

model22.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history22 = model22.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 50,
    verbose = 1
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 123, 500)          500000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 116, 64)           256064    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 58, 64)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 58, 64)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 58, 500)           32500     
_________________________________________________________________
dropout_4 (Dropout)          (None, 58, 500)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 58, 50)           

KeyboardInterrupt: 

In [27]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = 123
    ),
    tf.keras.layers.Conv1D(
        128,
        5,
        activation = 'relu'
    ),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(
        54,
        5,
        activation = 'relu'
    ),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(
        27,
        5,
        activation = 'relu'
    ),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(54, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(9, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model3.summary()
model3.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history3 = model3.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 50,
    verbose = 1
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 123, 500)          500000    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 119, 128)          320128    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 59, 128)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 55, 54)            34614     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 27, 54)            0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 27, 54)           

KeyboardInterrupt: 

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = SEQUENCES_LENGTH
    ),
    tf.keras.layers.Conv1D(
        32,
        6,
        activation = 'relu'
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer = Adam(lr=0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

historymodel_lstm_adam = model.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Epoch 1/5
1156/1156 [==============================] - 181s 155ms/step - loss: 0.4086 - accuracy: 0.8030 - val_loss: 0.2303 - val_accuracy: 0.9045
Epoch 2/5
1156/1156 [==============================] - 168s 145ms/step - loss: 0.2672 - accuracy: 0.8812 - val_loss: 0.2312 - val_accuracy: 0.9106
Epoch 3/5
1156/1156 [==============================] - 163s 141ms/step - loss: 0.2323 - accuracy: 0.9009 - val_loss: 0.2415 - val_accuracy: 0.9014
Epoch 4/5
1156/1156 [==============================] - 186s 161ms/step - loss: 0.1986 - accuracy: 0.9180 - val_loss: 0.2760 - val_accuracy: 0.8942
Epoch 5/5
1156/1156 [==============================] - 180s 156ms/step - loss: 0.1636 - accuracy: 0.9356 - val_loss: 0.2943 - val_accuracy: 0.8968


In [11]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = SEQUENCES_LENGTH
    ),
    tf.keras.layers.Conv1D(
        32,
        6,
        activation = 'relu'
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model2.compile(
    optimizer = Adam(lr=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

historymodel_lstm_adam = model2.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Epoch 1/5
1156/1156 [==============================] - 145s 125ms/step - loss: 0.5288 - accuracy: 0.7362 - val_loss: 0.2681 - val_accuracy: 0.8870
Epoch 2/5
1156/1156 [==============================] - 169s 146ms/step - loss: 0.3429 - accuracy: 0.8487 - val_loss: 0.2478 - val_accuracy: 0.9014
Epoch 3/5
1156/1156 [==============================] - 171s 148ms/step - loss: 0.2949 - accuracy: 0.8746 - val_loss: 0.2455 - val_accuracy: 0.9029
Epoch 4/5
1156/1156 [==============================] - 183s 158ms/step - loss: 0.2775 - accuracy: 0.8829 - val_loss: 0.2407 - val_accuracy: 0.9076
Epoch 5/5
1156/1156 [==============================] - 178s 154ms/step - loss: 0.2709 - accuracy: 0.8877 - val_loss: 0.2411 - val_accuracy: 0.9045


In [ ]:
from tensorflow.keras.optimizers import Adam
modelbaru = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        100,
        input_length = SEQUENCES_LENGTH
    ),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128)
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

modelbaru.compile(
    optimizer = Adam(lr=0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

historymodel_lstm_adam = modelbaru.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 7,
    verbose = 1
)

Epoch 1/7
  19/1156 [..............................] - ETA: 7:25 - loss: 0.6087 - accuracy: 0.7253

In [12]:
from tensorflow.keras.optimizers import Adam
modelbaru2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = SEQUENCES_LENGTH
    ),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(16)
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

modelbaru2.compile(
    optimizer = Adam(lr=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

historymodel_lstm_adam = modelbaru2.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Epoch 1/5
1156/1156 [==============================] - 292s 246ms/step - loss: 0.5184 - accuracy: 0.7448 - val_loss: 0.2561 - val_accuracy: 0.8942
Epoch 2/5
1156/1156 [==============================] - 267s 231ms/step - loss: 0.3095 - accuracy: 0.8653 - val_loss: 0.2475 - val_accuracy: 0.9024
Epoch 3/5
1156/1156 [==============================] - 272s 235ms/step - loss: 0.2822 - accuracy: 0.8806 - val_loss: 0.2460 - val_accuracy: 0.9040
Epoch 4/5
1156/1156 [==============================] - 268s 231ms/step - loss: 0.2702 - accuracy: 0.8837 - val_loss: 0.2427 - val_accuracy: 0.9019
Epoch 5/5
1156/1156 [==============================] - 275s 238ms/step - loss: 0.2616 - accuracy: 0.8873 - val_loss: 0.2453 - val_accuracy: 0.8978


In [31]:
saved_model_path = './model3/model_sentiment.h5'
modelbaru.save(saved_model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(modelbaru)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\MUHAMM~1\AppData\Local\Temp\tmptpjge_io\assets


INFO:tensorflow:Assets written to: C:\Users\MUHAMM~1\AppData\Local\Temp\tmptpjge_io\assets


In [ ]:
import pathlib
converter = tf.lite.TFLiteConverter.from_saved_model('./model1.h5')
tflite_model = converter.convert()
tflite_model_file = pathlib.Path('./')
tflite_model_file.write_bytes(tflite_model)

In [13]:
modelbaru2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = SEQUENCES_LENGTH
    ),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(25, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

modelbaru2.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

historymodel_lstm_adam = modelbaru2.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Epoch 1/5
1155/1155 [==============================] - 163s 137ms/step - loss: 0.6057 - accuracy: 0.7143 - val_loss: 0.4490 - val_accuracy: 0.8690
Epoch 2/5
1155/1155 [==============================] - 162s 141ms/step - loss: 0.5961 - accuracy: 0.7188 - val_loss: 0.4405 - val_accuracy: 0.8690
Epoch 3/5
1155/1155 [==============================] - 169s 146ms/step - loss: 0.5966 - accuracy: 0.7181 - val_loss: 0.4487 - val_accuracy: 0.8690
Epoch 4/5
1155/1155 [==============================] - 163s 141ms/step - loss: 0.5944 - accuracy: 0.7201 - val_loss: 0.4671 - val_accuracy: 0.8690
Epoch 5/5
1155/1155 [==============================] - 165s 143ms/step - loss: 0.5939 - accuracy: 0.7209 - val_loss: 0.4716 - val_accuracy: 0.8695


In [ ]:
model_conv_sgd = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        VOCAB_SIZE,
        500,
        input_length = 123
    ),
    tf.keras.layers.Conv1D(
        128,
        5,
        activation = 'relu'
    ),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(10, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model_conv_sgd.compile(
    optimizer = 'sgd',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

history_conv_sgd = model_conv_sgd.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 50,
    verbose = 1
)

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 129),
    
    tf.keras.layers.Conv1D(64, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history_adam = model.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 129, 500)          500000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 122, 64)           256064    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 61, 64)            0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)               

In [23]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 129),
    
    tf.keras.layers.Conv1D(64, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model2.summary()

model2.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history_adam = model2.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 129, 500)          500000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 122, 64)           256064    
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 61, 64)            0         
_________________________________________________________________
global_average_pooling1d_4 ( (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)               

In [32]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 129),
    
    tf.keras.layers.Conv1D(32, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model3.summary()

model3.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history_adam = model3.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 129, 500)          500000    
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 122, 32)           128032    
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 61, 32)            0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 61, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1952)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              1999872   
_________________________________________________________________
dropout_30 (Dropout)         (None, 1024)            

In [ ]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 500, input_length = 129),
    
    tf.keras.layers.Conv1D(16, 8, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(24, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model3.summary()

model3.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

history_adam = model3.fit(
    train_data, train_label,
    validation_data = (val_data, val_label),
    epochs = 5,
    verbose = 1
)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 129, 500)          500000    
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 122, 16)           64016     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 61, 16)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 976)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 24)                23448     
_________________________________________________________________
dropout_33 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)               

In [ ]:
train_databatch = tf.data.Dataset.from_tensor_slices((train_data, train_label))
train_databatch = train_databatch.batch(35).shuffle(1000).repeat()
print(train_databatch)